In [95]:
import pandas as pd
import numpy as np

import re
import os

### loop through all files

### define function to clean file names

In [96]:
coursename_semester = np.nan #global variable that can be used outside the function

def get_course_name(input_file):
    global coursename_semester #otherwise function will not overwrite
    
    filename = input_file.split(".csv")[0] # remove "".csv"

    # assign BSc / MSc
    if "BSc" in filename:
        bachelor_or_master = "BSc"
    else:
        bachelor_or_master = "MSc"

    # remove the irrelevant info before the course name
    # https://regex101.com/r/uEiYb2/1
    # https://www.debuggex.com/cheatsheet/regex/python
    coursename_semester = re.split(
        "(^.*(BSc|MSc)_(.._)?(._|.\d_\d_)?)", 
        filename)[-1]

    # assign nachtermin (based on "(digit)" in filename). doesnt work if multiple files with (digit) in filename
    # coursename_semester = re.sub(" \(\d\)", " NT", coursename_semester)

    # remove the WS/SS infor
    if "WS" in coursename_semester:
        coursename_semester = coursename_semester.split("_WS")
    elif "SS" in coursename_semester:
        coursename_semester = coursename_semester.split("_SS")

    # add the year after an @
    coursename_semester = "@".join(coursename_semester) + ";" + bachelor_or_master

    # print(coursename_semester)

## create basic df containing all students

In [ ]:
# set folder
dir = r"X:\BWL IV\Talentprogramm\2_Bestenlisten\Flexnow Listen\18_22"
 
# create an empty pandas data frame
df_baseline = pd.DataFrame()

# iterate over all .csv files within folder
for index, file in enumerate(os.listdir(dir)):
    if file.endswith(".csv"):
        df_file = pd.read_csv(os.path.join(dir, file), sep=";", decimal=",", encoding='unicode_escape')
        index_columns = ['Matrnr', 'Nachname', "Vorname", "Email"]
        df_file = df_file[index_columns]
        df_baseline = pd.concat([df_baseline , df_file], axis=0)

df_baseline.drop_duplicates(keep="first", inplace=True) # drop duplicates
df_baseline.reset_index(drop=True, inplace=True) # reset the index 

df_baseline

## merge all grades to baseline df

In [ ]:
# set folder
dir = r"X:\BWL IV\Talentprogramm\2_Bestenlisten\Flexnow Listen\18_22"
 
# create an empty pandas data frame
df_complete = pd.DataFrame()

# iterate over all .csv files within folder
for index, file in enumerate(os.listdir(dir)):
    if file.endswith(".csv"):

        # get clean course name from filename
        get_course_name(file)

        # load file into df and append filename to "Note" and "Fachsemester" columns
        df_file = pd.read_csv(os.path.join(dir, file), sep=";", decimal=",", encoding='unicode_escape') #will be overwritten in the next iteration

        # remove irrelevant columns
        irrelevant_columns = ["Nachname", "Vorname", "Email", 'Bemerkung','Versuch','Notentabelle','Prfdatum','Studienfach','Studienabschnitt','Studienabschnittsversion','Fachgruppe']
        df_file.drop(columns=irrelevant_columns, axis=1, inplace=True)
        df_file.dropna(how='all', axis=1, inplace=True) # there are some files with empty columns

        # set the first 4 columns as index (identifier) columns, which are ignored by add_suffix
        index_columns = ['Matrnr']
        df_file = df_file.set_index(index_columns)

        # add course name from filename to note and fachsemester columns
        df_file = df_file.add_suffix(f",{coursename_semester}")
        df_file = df_file.reset_index()

        # merge course file infos with df baseline
        df_baseline = df_baseline.merge(df_file, how="left", on=["Matrnr"])
print("\ndone")

df_baseline[:5]

## create a compact df

### grade average

In [ ]:
pd.set_option('mode.chained_assignment',None)

# drop columns containing "Fachsemester"
df_grade = df_baseline[df_baseline.columns.drop(list(df_baseline.filter(regex='Fachsemester')))]
# df_grade = df_baseline.filter(regex='|'.join(index_columns+["Note"]), axis=1)

# slice df down to grade columns
column_loc_after_mail = df_grade.columns.get_loc("Email") + 1 #grades start after Email column
df_grade.loc[:, "total average"] = df_grade.iloc[:, column_loc_after_mail:].mean(axis=1)
df_grade.loc[:, "received grades"] = df_grade.iloc[:, column_loc_after_mail:-1].count(axis=1) #ignore the newly inserted column
df_grade.loc[:, "max grade"] = df_grade.iloc[:, column_loc_after_mail:-2].max(axis=1)
df_grade.loc[:, "min grade"] = df_grade.iloc[:, column_loc_after_mail:-3].min(axis=1)

# drop all Note columns
df_grade_overview = df_grade[df_grade.columns.drop(list(df_grade.filter(regex='Note')))]

df_grade_overview.loc[:, "KoMa"] = df_grade.filter(regex="Kostenmanagement|Instruments", axis=1).max(axis=1)
df_grade_overview.loc[:, "Anwendungen"] = df_grade.filter(regex="Anwendungen|Systems", axis=1).max(axis=1)
df_grade_overview.loc[:, "Advanced"] = df_grade.filter(regex="Advanced", axis=1).max(axis=1)
df_grade_overview.loc[:, "Accounting"] = df_grade.filter(regex="Accounting_W", axis=1).max(axis=1)
df_grade_overview.loc[:, "Proseminar"] = df_grade.filter(regex="_Proseminar_", axis=1).max(axis=1)
df_grade_overview.loc[:, "Seminar"] = df_grade.filter(regex="_Seminar_", axis=1).max(axis=1)

df_grade_overview[:5]

### get the highest fachsemester and the date of the most recent klausur

In [100]:
## bachelor max semester##
# drop columns containing "Fachsemester"
df_bachelor_semester = df_baseline[df_baseline.columns.drop(list(df_baseline.filter(regex='Note|MSc')))]

# get max semester
df_bachelor_semester.loc[:, "max semester BSc"] = df_bachelor_semester.iloc[:, column_loc_after_mail:].max(axis=1)
# insert helper column to retrieve most recent semester
df_bachelor_semester.loc[:, "most recent semester BSc"] = df_bachelor_semester.iloc[:, column_loc_after_mail:].idxmax(axis=1)

df_bachelor_semester_relevant = df_bachelor_semester[["Matrnr", "Nachname", "Vorname", "Email", "max semester BSc", "most recent semester BSc"]]
df_bachelor_semester_relevant

# retrieve most recent semester
df_bachelor_semester_relevant["most recent semester BSc"] = df_bachelor_semester_relevant["most recent semester BSc"].str.extract('(\d\d_?\d?\d?)')

# df_bachelor_semester_relevant

## master max semester ##
# drop columns containing "Fachsemester"
df_master_semester = df_baseline[df_baseline.columns.drop(list(df_baseline.filter(regex='Note|BSc')))]

# get max semester
df_master_semester.loc[:, "max semester MSc"] = df_master_semester.iloc[:, column_loc_after_mail:].max(axis=1)
# insert helper column to retrieve most recent semester
df_master_semester.loc[:, "most recent semester MSc"] = df_master_semester.iloc[:, column_loc_after_mail:].idxmax(axis=1)

df_master_semester["MasterStudent"] = np.where(df_master_semester["max semester MSc"].isna() == False, 1, 0)

# only grab relevant columns
df_master_semester_relevant = df_master_semester[["Matrnr", "Nachname", "Vorname", "Email", "MasterStudent", "max semester MSc", "most recent semester MSc"]]

# retrieve most recent semester
df_master_semester_relevant["most recent semester MSc"] = df_master_semester_relevant["most recent semester MSc"].str.extract('(\d\d_?\d?\d?)')

# df_master_semester_relevant[df_master_semester_relevant["MasterStudent"] == 1]

In [101]:
df_final = df_grade_overview.merge(df_bachelor_semester_relevant, how="left", on=["Matrnr", "Nachname", "Vorname", "Email"])
df_final = df_final.merge(df_master_semester_relevant, how="left", on=["Matrnr", "Nachname", "Vorname", "Email"])

df_final.sort_values(by=["total average", "received grades"], ascending=[False, False], inplace=True)

# print(df_final.columns.tolist())
df_final = df_final[['Matrnr', 'Nachname', 'Vorname', 'Email', 'MasterStudent', 'max semester BSc', 'most recent semester BSc', 'max semester MSc', 'most recent semester MSc', 'total average', 'received grades', 'max grade', 'min grade', 'KoMa', 'Anwendungen', 'Advanced', 'Accounting', 'Seminar']]

df_final.reset_index(drop=True, inplace=True)
df_final[:5]

,Matrnr,Nachname,Vorname,Email,MasterStudent,max semester BSc,most recent semester BSc,max semester MSc,most recent semester MSc,total average,received grades,max grade,min grade,KoMa,Anwendungen,Advanced,Accounting,Seminar
0,3081099,Hormel,Henri Leonard,henri-leonard.hormel@wirtschaft.uni-giessen.de,0,5.0,21_22,NaN,NaN,15.0,3,15.0,15.0,NaN,NaN,NaN,NaN,NaN
1,1044695,Buz,Maxim,Maxim.Buz@wirtschaft.uni-giessen.de,0,7.0,19_20,NaN,NaN,15.0,2,15.0,15.0,NaN,NaN,NaN,NaN,NaN
2,4049497,Metternich,Jan Vincent,Jan.V.Metternich@wirtschaft.uni-giessen.de,0,7.0,19_20,NaN,NaN,15.0,2,15.0,15.0,NaN,NaN,NaN,NaN,NaN
3,3001097,Koller,Annika,annika.koller@wirtschaft.uni-giessen.de,0,5.0,20_21,NaN,NaN,15.0,2,15.0,15.0,NaN,NaN,NaN,NaN,NaN
4,5019699,Stoll,Linus Joshua,linus-joshua.stoll@wirtschaft.uni-giessen.de,0,4.0,20_21,NaN,NaN,15.0,2,15.0,15.0,NaN,NaN,NaN,NaN,NaN


In [102]:
df_baseline.to_csv(r"Student_Grades_18to22_complete.csv", sep=";", decimal=",")
df_final.to_csv(r"Student_Grades_18to22_compact.csv", sep=";", decimal=",")